<a href="https://colab.research.google.com/github/hansjang/Py_Radar/blob/main/finance1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# 네이버 뉴스 수집해서 엑셀로 저장하기
# !pip install bs4 requests
import requests
from bs4 import BeautifulSoup

# !pip install openpyxl
from openpyxl import Workbook

from datetime import datetime

def get_news(keyword):
    wb= Workbook()
    sheet = wb.active

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
        'Accept-Language': 'en-US,en;q=0.5',
        'Referer': 'https://www.naver.com/'
    }
    url = f'https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query={keyword}'

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching data: {e}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')

    # Updated CSS selector to accurately select news articles
    lis = soup.select('div.group_news > ul > li')

    if not lis:
        print("No news articles found.")
        return

    for li in lis:
        a = li.select_one('a.news_tit')
        row = [a.text, a['href']]
        sheet.append(row)

    today = datetime.today().strftime("%Y-%m-%d")

    wb.save(f"news/{today}_{keyword}.xlsx")
    wb.close()



In [ ]:
# 다수 키워드 수집
keywords = ['삼성전자', '현대자동차', 'lg전자', 'sk', '기아', '두산중공업', '삼성물산', '삼성바이오로직스', '삼성생명', '삼성전기']

for keyword in keywords:
  print(keyword)
  get_news(keyword)

In [24]:
# 파일명 바꾸기
import os

path = '/content/news'
files = os.listdir(path)

for name in files:
  new_name = name.split('.')[0]+'(뉴스).xlsx'
  os.rename(f'/content/news/{name}', f'/content/news/{new_name}')

In [29]:
# 엑셀파일에 저장된 주식 코드 불러와서 이미지 다운받기
import urllib.request

import openpyxl
wb = openpyxl.load_workbook('관리종목2024.xlsx')
sheet = wb['종목']

new_rows = list(sheet.rows)[1:]

for row in new_rows:
  code = row[1].value
  name = row[0].value
  url = f'https://ssl.pstatic.net/imgfinance/chart/item/area/year3/{code}.png'
  urllib.request.urlretrieve(url, f"imgs/{name}.png")

In [ ]:
# 압축해서 다운받기
!zip -r /content/files.zip /content/imgs